In [17]:
%reload_ext autoreload
%autoreload 2

In [18]:
RESULTS_PATH = '../results/'

In [137]:
import numpy as np
import inputs
import classification
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals import joblib

Select the parameters and models for training, validation and testing

In [225]:
task_list = [inputs.binary]#[inputs.binary, inputs.six_transient, inputs.seven_transient, inputs.seven_class, inputs.eight_class]
min_obs_list = [10]
num_features_list = [30]
oversample_list = [True]
model_list = [classification.rf]#classification.svc, classification.rf, classification.mlp]
scaler_list = [StandardScaler]#, MinMaxScaler]

In [226]:
# task_list = [inputs.binary, inputs.six_transient, inputs.seven_transient, inputs.seven_class, inputs.eight_class]
# min_obs_list = [5,10]
# num_features_list = [30, 26, 20]
# oversample_list = [True, False]
# model_list = [classification.svc, classification.rf, classification.mlp]
# scaler_list = [StandardScaler, MinMaxScaler]

For every combination of parameters selected, perform pre-processing and classification

In [227]:
def combination_results_dict(min_obs, num_features, oversample, model_name, scaler_name, best_score, best_params):
    combination_results = {}
    combination_results['min_obs'] = min_obs
    combination_results['num_features'] = num_features
    combination_results['oversample'] = oversample
    combination_results['model'] = model_name
    combination_results['scaler'] = scaler_name
    combination_results['best_score']  = best_score
    combination_results['best_params'] = best_params
    return combination_results

In [233]:
combinations = itertools.product(
    min_obs_list, num_features_list, oversample_list, model_list, scaler_list
)
for task in task_list:
    results_task = list()
    results_models = list()
    for combination in combinations:
        
        min_obs, num_features, oversample, model, scaler = combination      
        print('STARTING TASK: ', task.__name__, min_obs, num_features, oversample, model.__name__, scaler.__name__)

        # Obtain inputs
        X_train, X_test, y_train, y_test = task(min_obs, num_features,
                                                oversample=oversample)
        # Scale inputs
        fit_scaler = scaler().fit(X_train)
        X_train = fit_scaler.transform(X_train); X_test = fit_scaler.transform(X_test)
        # Perform Classification
        clf = model(X_train, y_train, X_test, y_test, 
                    min_obs, num_features, oversample, task, fit_scaler)
    
        # Store temporary results
        results_models.append(clf.best_estimator_)
        
        combination_results = combination_results_dict(
            min_obs, num_features, oversample, model.__name__,
            scaler.__name__, clf.best_score_,clf.best_params_
        )
        results_task.append(combination_results)
        
    # Generate DataFrame with current task's results
    df_results_task = pd.DataFrame(results_task)
    df_results_task.to_pickle(RESULTS_PATH + 'dataframes/{}.pkl'.format(task.__name__))
    
    # Find and Save best model
    task_best_index = df_results_task['best_score'].idxmax()
    task_best_model = results_models[task_best_index]
    joblib.dump(task_best_model, RESULTS_PATH + 'models/{}.pkl'.format(task.__name__))

STARTING TASK:  binary 10 30 True rf StandardScaler
[[  1.12135      0.30769231   0.06736346   0.14507436   0.15970023
    0.29611975   0.51916946  -0.55294691   7.12407381   0.54422249
    0.07692308   0.46153846   0.2         21.04956368  12.48804478
    0.32114705  -4.46691107   4.94277932   3.78776031  -9.86318526
    6.81205335 -21.78511289  47.63374    -37.46448387  12.34726829
   -0.28759767  -3.92727273   0.6814712   -0.56282221   0.70797157]]
[[ 1.00000000e-01  3.61111111e-01  9.48667925e-02  2.38112247e-01
   3.13897330e-01  5.43428216e-01  6.99187232e-01  2.13463856e-01
   7.81738587e-01  2.97205075e-02  7.40740741e-01  4.44444444e-01
   4.33333333e-01  2.58925412e-01  1.13079364e-01 -4.16497732e-02
   7.23910155e-03 -4.91592955e-02  1.70182451e-02 -1.91722911e-02
  -3.84444639e-02 -9.13971379e-01  1.84676984e+00 -1.17428879e+00
   2.02709401e-01  1.35691819e-01 -3.08598383e+00  3.83660522e-02
   6.63615272e-02  7.79695561e-01]
 [ 1.63500000e+00  2.00000000e-01  1.01162511e-

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'double'

In [223]:
df_results_task

,best_params,best_score,min_obs,model,num_features,oversample,scaler
0,"{'max_features': 'auto', 'n_estimators': 200}",0.742822,10,rf,30,False,StandardScaler
